### Задание теме «Коллаборативная фильтрация»

In [1]:
from surprise import KNNWithMeans, KNNBasic, KNNBaseline
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate

import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [3]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [4]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [5]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [6]:
movies_with_ratings[movies_with_ratings.userId == 2.0].title.unique()

array(['Shawshank Redemption, The (1994)', 'Tommy Boy (1995)',
       'Good Will Hunting (1997)', 'Gladiator (2000)',
       'Kill Bill: Vol. 1 (2003)', 'Collateral (2004)',
       'Talladega Nights: The Ballad of Ricky Bobby (2006)',
       'Departed, The (2006)', 'Dark Knight, The (2008)',
       'Step Brothers (2008)', 'Inglourious Basterds (2009)',
       'Zombieland (2009)', 'Shutter Island (2010)',
       'Exit Through the Gift Shop (2010)', 'Inception (2010)',
       'Town, The (2010)', 'Inside Job (2010)',
       'Louis C.K.: Hilarious (2010)', 'Warrior (2011)',
       'Dark Knight Rises, The (2012)',
       'Girl with the Dragon Tattoo, The (2011)',
       'Django Unchained (2012)', 'Wolf of Wall Street, The (2013)',
       'Interstellar (2014)', 'Whiplash (2014)', 'The Drop (2014)',
       'Ex Machina (2015)', 'Mad Max: Fury Road (2015)',
       'The Jinx: The Life and Deaths of Robert Durst (2015)'],
      dtype=object)

In [7]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [8]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [9]:
ratings.rating.min()

0.5

In [10]:
ratings.rating.max()

5.0

In [11]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [12]:
#  SVD


In [13]:
modelSVD = SVD()

In [14]:
res = cross_validate(modelSVD, data, measures=['RMSE'], cv=5, verbose=True)
res

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8751  0.8631  0.8703  0.8794  0.8755  0.8727  0.0056  
Fit time          3.61    3.58    3.56    3.58    3.58    3.58    0.02    
Test time         0.09    0.08    0.09    0.08    0.09    0.09    0.01    


{'test_rmse': array([0.87510761, 0.86314349, 0.87029501, 0.8793866 , 0.87551726]),
 'fit_time': (3.6122469902038574,
  3.577282428741455,
  3.5616304874420166,
  3.577307939529419,
  3.5772838592529297),
 'test_time': (0.09372544288635254,
  0.07810068130493164,
  0.09372806549072266,
  0.07809925079345703,
  0.0937032699584961)}

In [15]:
print(f"RMSE modelSVD: {np.mean(res['test_rmse'])}")

RMSE modelSVD: 0.872689991449487


In [20]:
# надо меньше 0.87, попробуем другую модель

In [16]:
#  KNNBaseline


In [17]:
modelKNNBaseline = KNNBaseline(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})

In [18]:
res = cross_validate(modelKNNBaseline, data, measures=['RMSE'], cv=5)
res

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.85792496, 0.84800007, 0.86115233, 0.84751805, 0.84570955]),
 'fit_time': (9.085641860961914,
  8.982264041900635,
  9.156014919281006,
  9.601234912872314,
  9.519592523574829),
 'test_time': (6.08622407913208,
  6.239374160766602,
  6.197036266326904,
  6.613490581512451,
  6.213878393173218)}

In [19]:
print(f"RMSE modelKNNBaseline: {np.mean(res['test_rmse'])}")

RMSE modelKNNBaseline: 0.8520609897208479


In [ ]:
# бинго